In [1]:
import numpy as np
import matplotlib.pyplot as plt
from ORRFA import *
import interpretableai
from interpretableai import iai
import pandas as pd
import json
from HR import *

In [2]:
dataset = pd.read_csv("/Users/ryanlucas/Desktop/Steel_industry_data.csv")

train_idx = np.arange(0, int(0.2*len(dataset)))
test_idx = np.arange(int(0.2*len(dataset)), int(0.3*len(dataset)))

features = dataset.loc[train_idx].iloc[:, 2:-4]
diagnosis = dataset.loc[train_idx].iloc[:, 1]

orig_features_size = features.copy().shape[1]

In [3]:
features_test = dataset.loc[test_idx].iloc[:, 2:-4]
diagnosis_test = dataset.loc[test_idx].iloc[:, 1]

In [4]:
diagnosis

0       3.17
1       4.00
2       3.24
3       3.31
4       3.82
        ... 
7003    4.07
7004    3.96
7005    3.96
7006    3.96
7007    4.03
Name: Usage_kWh, Length: 7008, dtype: float64

In [5]:
print(len(dataset))
print(len(features))
print(len(features_test))

35040
7008
3504


In [6]:
features

,Lagging_Current_Reactive.Power_kVarh,Leading_Current_Reactive_Power_kVarh,CO2(tCO2),Lagging_Current_Power_Factor,Leading_Current_Power_Factor
0,2.95,0.0,0.0,73.21,100.0
1,4.46,0.0,0.0,66.77,100.0
2,3.28,0.0,0.0,70.28,100.0
3,3.56,0.0,0.0,68.09,100.0
4,4.50,0.0,0.0,64.72,100.0
...,...,...,...,...,...
7003,6.48,0.0,0.0,53.19,100.0
7004,5.65,0.0,0.0,57.39,100.0
7005,5.76,0.0,0.0,56.65,100.0
7006,5.58,0.0,0.0,57.87,100.0


In [7]:
def gen_features(sub_paths, data, features, orig_feature_size):
    new_features = features.copy()
    for path in sub_paths:
        statement = gen_statement(path, data, features)
        if statement != '(0 < 0.0)' and statement != '(0 >= 0.0)':

            try:
                index_feature = features.loc[eval(statement)].index


                new_feature = [1 if i in index_feature else 0 for i in range(len(features))]

                if sum(new_feature) >= 1:
                    new_features[statement] = new_feature

            except:
                print(statement)
    return new_features.iloc[:, orig_feature_size:]

In [8]:
np.random.seed(1)
n = 5
rules = {}
for i in range(1,15):

    #################
    sampled_features = features.copy().sample(n=int(n),axis='columns')

    OCT_H = iai.GridSearch(
          iai.OptimalTreeRegressor(
              max_depth = 3, cp = 0.00000000001))

    OCT_H.fit(sampled_features, diagnosis)
    OCT_H.write_json(f"tre_{i}.json")

    ######################
    f = open(f"tre_{i}.json")
    data = json.load(f)

    paths = gen_paths(data)

    rule_features_new = gen_features(
            paths, data, features, orig_features_size)

    rules[f"tre_{i}"] = rule_features_new

    print(f"Tree: {i}")


[ Warning: This copy of Interpretable AI software is for academic purposes only and not for commercial use.


Tree: 1
Tree: 2
Tree: 3
Tree: 4
Tree: 5
Tree: 6
Tree: 7
Tree: 8
Tree: 9
Tree: 10
Tree: 11
Tree: 12
Tree: 13
Tree: 14


In [9]:
features.columns

Index(['Lagging_Current_Reactive.Power_kVarh',
       'Leading_Current_Reactive_Power_kVarh', 'CO2(tCO2)',
       'Lagging_Current_Power_Factor', 'Leading_Current_Power_Factor'],
      dtype='object')

In [10]:
big_df = pd.DataFrame()
for tree in rules.keys():

    big_df = pd.concat([big_df, rules[tree]],axis =1)


In [11]:
df = big_df.loc[:,~big_df.columns.duplicated()].copy()

In [12]:
df

,(features['Lagging_Current_Reactive.Power_kVarh'] >= 0.045) & (features['Lagging_Current_Reactive.Power_kVarh'] >= 0.05499999999999999) & (0 >= 0.0),(features['Leading_Current_Reactive_Power_kVarh'] < 0.005) & (features['Lagging_Current_Reactive.Power_kVarh'] < 22.714999999999996) & (0 >= 0.0),(features['Leading_Current_Reactive_Power_kVarh'] >= 0.045) & (features['Leading_Current_Reactive_Power_kVarh'] >= 0.05499999999999999) & (0 >= 0.0),(features['Lagging_Current_Reactive.Power_kVarh'] < 0.005) & (features['Leading_Current_Power_Factor'] < 22.714999999999996) & (0 >= 0.0),(features['Lagging_Current_Power_Factor'] >= 0.045) & (features['Lagging_Current_Power_Factor'] >= 0.05499999999999999) & (0 >= 0.0),(features['Leading_Current_Reactive_Power_kVarh'] < 0.005) & (features['CO2(tCO2)'] < 22.714999999999996) & (0 >= 0.0),(features['CO2(tCO2)'] < 0.005) & (features['Leading_Current_Reactive_Power_kVarh'] < 22.714999999999996) & (0 >= 0.0),(features['CO2(tCO2)'] >= 0.045) & (features['CO2(tCO2)'] >= 0.05499999999999999) & (0 >= 0.0),(features['Lagging_Current_Reactive.Power_kVarh'] < 0.005) & (features['Leading_Current_Reactive_Power_kVarh'] < 22.714999999999996) & (0 >= 0.0),(features['CO2(tCO2)'] < 0.005) & (features['Lagging_Current_Reactive.Power_kVarh'] < 22.714999999999996) & (0 >= 0.0)
0,1,1,0,0,1,1,1,0,0,1
1,1,1,0,0,1,1,1,0,0,1
2,1,1,0,0,1,1,1,0,0,1
3,1,1,0,0,1,1,1,0,0,1
4,1,1,0,0,1,1,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...
7003,1,1,0,0,1,1,1,0,0,1
7004,1,1,0,0,1,1,1,0,0,1
7005,1,1,0,0,1,1,1,0,0,1
7006,1,1,0,0,1,1,1,0,0,1


In [13]:
for column in df.columns[1:]:
    if df[column].sum() == len(df):
        del df[column]


In [14]:
df

,(features['Lagging_Current_Reactive.Power_kVarh'] >= 0.045) & (features['Lagging_Current_Reactive.Power_kVarh'] >= 0.05499999999999999) & (0 >= 0.0),(features['Leading_Current_Reactive_Power_kVarh'] < 0.005) & (features['Lagging_Current_Reactive.Power_kVarh'] < 22.714999999999996) & (0 >= 0.0),(features['Leading_Current_Reactive_Power_kVarh'] >= 0.045) & (features['Leading_Current_Reactive_Power_kVarh'] >= 0.05499999999999999) & (0 >= 0.0),(features['Lagging_Current_Reactive.Power_kVarh'] < 0.005) & (features['Leading_Current_Power_Factor'] < 22.714999999999996) & (0 >= 0.0),(features['Leading_Current_Reactive_Power_kVarh'] < 0.005) & (features['CO2(tCO2)'] < 22.714999999999996) & (0 >= 0.0),(features['CO2(tCO2)'] < 0.005) & (features['Leading_Current_Reactive_Power_kVarh'] < 22.714999999999996) & (0 >= 0.0),(features['CO2(tCO2)'] >= 0.045) & (features['CO2(tCO2)'] >= 0.05499999999999999) & (0 >= 0.0),(features['Lagging_Current_Reactive.Power_kVarh'] < 0.005) & (features['Leading_Current_Reactive_Power_kVarh'] < 22.714999999999996) & (0 >= 0.0),(features['CO2(tCO2)'] < 0.005) & (features['Lagging_Current_Reactive.Power_kVarh'] < 22.714999999999996) & (0 >= 0.0)
0,1,1,0,0,1,1,0,0,1
1,1,1,0,0,1,1,0,0,1
2,1,1,0,0,1,1,0,0,1
3,1,1,0,0,1,1,0,0,1
4,1,1,0,0,1,1,0,0,1
...,...,...,...,...,...,...,...,...,...
7003,1,1,0,0,1,1,0,0,1
7004,1,1,0,0,1,1,0,0,1
7005,1,1,0,0,1,1,0,0,1
7006,1,1,0,0,1,1,0,0,1


In [15]:
features_for_robust_reg = df

In [16]:
rule_features = features_for_robust_reg.copy()
features_for_robust_reg = pd.concat([features, features_for_robust_reg], axis = 1)

In [17]:
features_for_robust_reg.sum()

Lagging_Current_Reactive.Power_kVarh                                                                                                                    103483.07
Leading_Current_Reactive_Power_kVarh                                                                                                                     29470.59
CO2(tCO2)                                                                                                                                                  107.82
Lagging_Current_Power_Factor                                                                                                                            603676.42
Leading_Current_Power_Factor                                                                                                                            604126.91
(features['Lagging_Current_Reactive.Power_kVarh'] >= 0.045) & (features['Lagging_Current_Reactive.Power_kVarh'] >= 0.05499999999999999) & (0 >= 0.0)      5701.00
(features['Leading_Current_R

In [18]:
features_for_robust_reg.insert(loc = 0, column = 'Intercept', value = [1 for i in range(len(features_for_robust_reg))])

In [19]:
features_for_robust_reg

,Intercept,Lagging_Current_Reactive.Power_kVarh,Leading_Current_Reactive_Power_kVarh,CO2(tCO2),Lagging_Current_Power_Factor,Leading_Current_Power_Factor,(features['Lagging_Current_Reactive.Power_kVarh'] >= 0.045) & (features['Lagging_Current_Reactive.Power_kVarh'] >= 0.05499999999999999) & (0 >= 0.0),(features['Leading_Current_Reactive_Power_kVarh'] < 0.005) & (features['Lagging_Current_Reactive.Power_kVarh'] < 22.714999999999996) & (0 >= 0.0),(features['Leading_Current_Reactive_Power_kVarh'] >= 0.045) & (features['Leading_Current_Reactive_Power_kVarh'] >= 0.05499999999999999) & (0 >= 0.0),(features['Lagging_Current_Reactive.Power_kVarh'] < 0.005) & (features['Leading_Current_Power_Factor'] < 22.714999999999996) & (0 >= 0.0),(features['Leading_Current_Reactive_Power_kVarh'] < 0.005) & (features['CO2(tCO2)'] < 22.714999999999996) & (0 >= 0.0),(features['CO2(tCO2)'] < 0.005) & (features['Leading_Current_Reactive_Power_kVarh'] < 22.714999999999996) & (0 >= 0.0),(features['CO2(tCO2)'] >= 0.045) & (features['CO2(tCO2)'] >= 0.05499999999999999) & (0 >= 0.0),(features['Lagging_Current_Reactive.Power_kVarh'] < 0.005) & (features['Leading_Current_Reactive_Power_kVarh'] < 22.714999999999996) & (0 >= 0.0),(features['CO2(tCO2)'] < 0.005) & (features['Lagging_Current_Reactive.Power_kVarh'] < 22.714999999999996) & (0 >= 0.0)
0,1,2.95,0.0,0.0,73.21,100.0,1,1,0,0,1,1,0,0,1
1,1,4.46,0.0,0.0,66.77,100.0,1,1,0,0,1,1,0,0,1
2,1,3.28,0.0,0.0,70.28,100.0,1,1,0,0,1,1,0,0,1
3,1,3.56,0.0,0.0,68.09,100.0,1,1,0,0,1,1,0,0,1
4,1,4.50,0.0,0.0,64.72,100.0,1,1,0,0,1,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7003,1,6.48,0.0,0.0,53.19,100.0,1,1,0,0,1,1,0,0,1
7004,1,5.65,0.0,0.0,57.39,100.0,1,1,0,0,1,1,0,0,1
7005,1,5.76,0.0,0.0,56.65,100.0,1,1,0,0,1,1,0,0,1
7006,1,5.58,0.0,0.0,57.87,100.0,1,1,0,0,1,1,0,0,1


In [22]:
model = HolisticRobust(
                       α = 0,
                       ϵ = 0,
                       r = 0,
                       classifier = "Linear",
                       learning_approach = "HR") # Could be ERM either

In [ ]:
θ, obj_value = model.fit(ξ = (np.matrix(features_for_robust_reg), np.array(diagnosis)))

In [ ]:
predictions = np.multiply(θ, features_for_robust_reg
                          ).sum(axis = 1)

In [ ]:
diagnosis.plot()

In [ ]:
(diagnosis - predictions).describe()

In [ ]:
SSE_model = sum((diagnosis - predictions)**2)
SSE_mean = sum((diagnosis - diagnosis.mean())**2)

1 - SSE_model/SSE_mean

In [ ]:
features

In [ ]:
rule_features

In [ ]:
features = features_test

for rule in rule_features.columns:

    index_feature = features.loc[eval(rule)].index

    new_feature = [1 if i in index_feature else 0 for i in range(len(features))]

    if sum(new_feature) >= 1:
        features[rule] = new_feature


In [ ]:
features